### Purpose

Auxiliary notebook for practical 1 (Thursday) of Week 1, exercises 2 and 3 on Euler method vs. analytical solutions.

### Imports

In [1]:
#import necessary packages
import pandas as pd #dataframes
import numpy as np #math and arrays
from scipy.integrate import odeint #differential equations
import matplotlib.pyplot as plt #plotting

### Part 2

#### Initial conditions

In [ ]:
# Define the simulation end time and number of time points for which you want output
t0 = 0
t_end = 1
h = 0.2  # this is delta t, the time step size
Nsteps = int((t_end - t0)/h)
print(f"Nsteps: {Nsteps}")

# Make a numpy array with the time steps for which you want output
timepoints = np.linspace(t0, t_end, Nsteps+1)  # +1 to include the endpoint
timepoints

Nsteps: 5


array([0. , 0.2, 0.4, 0.6, 0.8, 1. ])

In [3]:
# Set initial values for state variables
x1_init = 1
x_init = [x1_init] # combine into a list

# Parameters
k = -2
parameters = (k,)

#### Euler approach

In [4]:
# Euler method per time step
def euler(x, f, delta):
    x = x + delta * f(x)
    return x

def f(x):
    return -2 * x

In [ ]:
# Run the Euler method iteratively over the provided timesteps
x_euler = [x_init[0]]

for i in range(1, len(timepoints)):
    x_next = euler(x_euler[-1], f, delta=h)
    x_euler.append(x_next)
    
x_t_euler = np.array(x_euler)
x_t_euler

array([1.     , 0.6    , 0.36   , 0.216  , 0.1296 , 0.07776])

#### ODE approach

In [6]:
def deriv(x, t, k):
    
    # Unpack the state variable information
    # The first argument contains the values of the state variables in order
    x = x[0] 

    # Calculate the derivatives of the only state
    dxdt = k * x
    return dxdt

In [7]:
# Calculate state variables at requested time points
x_t_ode = odeint(deriv, x_init, timepoints, args = parameters)
x_t_ode

array([[1.        ],
       [0.67032006],
       [0.44932898],
       [0.30119422],
       [0.20189652],
       [0.13533527]])

#### Error computation

In [8]:
def point_wise_error(Nsteps, x_t_ode, x_t_euler):
    # Point-wise subtraction between x_t_ode and x_t_euler
    cum_squared_diff = (x_t_ode.flatten() - x_t_euler)**2
    return cum_squared_diff, (cum_squared_diff.sum())/Nsteps

In [9]:
cum_squared_diff, avg_error = point_wise_error(Nsteps, x_t_ode, x_t_euler)
print("Average error:", avg_error)

Average error: 0.005744866136517562


#### Results summary

In [10]:
results_dict = {
    "Timepoints": timepoints,
    "x_t_ode": x_t_ode.flatten(),
    "x_t_euler": x_t_euler,
    "cum_squared_diff": cum_squared_diff
}
results_df = pd.DataFrame(results_dict).round(4)
results_df

,Timepoints,x_t_ode,x_t_euler,cum_squared_diff
0,0.0,1.0000,1.0000,0.0000
1,0.2,0.6703,0.6000,0.0049
2,0.4,0.4493,0.3600,0.0080
3,0.6,0.3012,0.2160,0.0073
4,0.8,0.2019,0.1296,0.0052
5,1.0,0.1353,0.0778,0.0033


### Part 3

#### Euler approach

In [ ]:
# Define the simulation end time and number of time points for which you want output
t0 = 0
t_end = 3
h = 1  # this is delta t, the time step size
Nsteps = int((t_end - t0)/h)
print(f"Nsteps: {Nsteps}")

# Make a numpy array with the time steps for which you want output
timepoints = np.linspace(t0, t_end, Nsteps+1)  # +1 to include the endpoint
print(f"timepoints: {timepoints}")

Nsteps: 3
timepoints: [0. 1. 2. 3.]


In [12]:
X0 = 0
Y0 = 0
init = [X0, Y0]

a = 1
b = 0.1
c = 2
d = 0.5
parameters = (a, b, c, d)

In [13]:
# Euler method per time step for a system of equations
def euler_system(x, y, f, g, params, delta):
    a, b, c, d = params
    xnew = x + delta * f(x, a, b)
    ynew = y + delta * g(x, y, c, d)
    return [xnew, ynew]

def f(x, a, b):
    return a - b * x

def g(x, y, c, d):
    return c * x - d * y

In [ ]:
# Run the Euler method iteratively over the provided system
x_euler = [init]

for i in range(1, len(timepoints)):
    x_prev, y_prev = x_euler[-1]  # last state in the list
    x_next = euler_system(x_prev, y_prev, f, g, parameters, delta=h)
    x_euler.append(x_next)
    
x_t_euler = np.array(x_euler)
x_t_euler

array([[0.  , 0.  ],
       [1.  , 0.  ],
       [1.9 , 2.  ],
       [2.71, 4.8 ]])

#### Analytical expressions

In [15]:
def X(timepoints):
    return 10*(1 - np.exp(-0.1*timepoints))

def Y(timepoints):
    return 40 - 50 * np.exp(-0.1*timepoints) + 10 * np.exp(-0.5*timepoints)

In [16]:
x_t_analytical = np.array([X(timepoints), Y(timepoints)]).T
x_t_analytical

array([[0.        , 0.        ],
       [0.95162582, 0.8234357 ],
       [1.81269247, 2.74225676],
       [2.59181779, 5.19039057]])

#### Results summary for Y(t) and error

In [17]:
# Results summary for Y(t) and error
Y_analytical = x_t_analytical[:, 1]
Y_euler = x_t_euler[:, 1]
err = Y_analytical - Y_euler
cum_squared_diff_Y = (Y_analytical - Y_euler)**2

# As dataframe
results_dict_Y = {
    "Timepoints": timepoints,
    "Y_analytical": Y_analytical,
    "Y_euler": Y_euler,
    "err": err,
    "cum_squared_diff_Y": cum_squared_diff_Y
}
results_df_Y = pd.DataFrame(results_dict_Y).round(4)
results_df_Y

,Timepoints,Y_analytical,Y_euler,err,cum_squared_diff_Y
0,0.0,0.0000,0.0,0.0000,0.0000
1,1.0,0.8234,0.0,0.8234,0.6780
2,2.0,2.7423,2.0,0.7423,0.5509
3,3.0,5.1904,4.8,0.3904,0.1524


In [18]:
avg_error_Y = (cum_squared_diff_Y.sum())/Nsteps
print("Average error for Y:", avg_error_Y)

Average error for Y: 0.4604654113256905
